# カスタマーチャーンの予測モデルを作成しよう

以下の手順を実施していきます。
    0. 使用するライブラリの準備
    1. データを読み込む
    2. データを調べる
    3. 機械学習用にデータを準備する
    4. 学習用とテスト用にデータを分割する
    5. 二値分類に関す種々のアルゴリズムを用いて、モデルを作成する
    6. モデルを評価する
    7. 今回のデータに最も適したモデルを選ぶ
    8. （選んだモデルを保存、デプロイする）

## 0. 使用するライブラリの準備

本notebookでは機械学習モデル作成用にscikit-learnというオープンソースを使用する。
また、データを取り扱うためのライブラリであるpandasやnumpy、グラフ描画用のmatplotlibやseabornも使用していく。

まずはこれらのライブラリを本環境に読み込んでいく。

In [ ]:
#scikit-learnが提供しているアルゴリズムから、二値分類に関するものを読み込んでいく。
from sklearn import model_selection
from sklearn import tree
from sklearn import svm
from sklearn import ensemble
from sklearn import neighbors
from sklearn import linear_model
from sklearn import metrics
from sklearn import preprocessing

In [ ]:
%matplotlib inline 
#使用するライブラリを読み込んでいく。

from IPython.display import Image
import matplotlib as mlp
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import seaborn as sns
import json

## Dataset

今回はAutoAIでモデルを作成したときと同様、charn.csvを使用します。

※ こちらからもダウンロードできます。
https://www.kaggle.com/becksddf/churn-in-telecoms-dataset/data. 

アセットからinsert to codeをクリックし、下のセルに読み込むプログラムを追加します。

#下の行にカーソルを合わせ、insert to code > pandas dataframe

#以下の行の、df_data_Xの部分をご自身の値に変更してください（ex. df_data_1）

df = df_data_X
print (df.shape)

In [ ]:
# 下の行にカーソルを合わせ、insert to code > pandas dataframe

# 以下の行の、df_data_Xの部分をご自身の値に変更してください（ex. df_data_1）
df = df_data_X
print (df.shape)

読み込んだデータの先頭5行を見てみましょう。

In [ ]:
#データフレームの先頭５行を表示
#head(): 用意されている関数
    
df.head()

In [ ]:
y = df["churn"].value_counts()
sns.barplot(y.index, y.values)

In [ ]:
y_True = df["churn"][df["churn"] == True]
print ("Churn Percentage = "+str( (y_True.shape[0] / df["churn"].shape[0]) * 100 ))

## データの分析：Descriptive Analysis

特定の項目と、チャーンの発生率を見てみましょう。  
matplotlibのplot()関数を使用することにより、視覚的に関係を描画します。

In [ ]:
  df.describe()

### Stateとチャーンの関係 

In [ ]:
df.groupby(["state", "churn"]).size().unstack().plot(kind='bar', stacked=True, figsize=(30,10)) 

### Area Code とチャーンの関係

In [ ]:
df.groupby(["area code", "churn"]).size().unstack().plot(kind='bar', stacked=True, figsize=(5,5)) 

###  選択しているプランとチャーンの関係 

In [ ]:
df.groupby(["international plan", "churn"]).size().unstack().plot(kind='bar', stacked=True, figsize=(5,5)) 

### Voice mail planとチャーンの関係

In [ ]:
df.groupby(["voice mail plan", "churn"]).size().unstack().plot(kind='bar', stacked=True, figsize=(5,5)) 

## データの準備

機械学習モデルに取り込む最終的なデータセットを作成するために、以下の手順を実施していきます。
一般に、データを準備するためのこれらのタスクは順不同で複数回行われます。


1. データ変数のタイプを調整する
2. 応答ベクターを作成する
3. 余分な列を削除する
4. フィーチャーマトリクスを作成する
5. データを正規化する


### Encode categorical columns

In [ ]:
# Integerへのエンコーダーの読み込み
label_encoder = preprocessing.LabelEncoder()

# State, international plans,  voice mail planは文字列のデータのため、Int（数値）型に変換します。
df['state'] = label_encoder.fit_transform(df['state'])
df['international plan'] = label_encoder.fit_transform(df['international plan'])
df['voice mail plan'] = label_encoder.fit_transform(df['voice mail plan'])

print (df.dtypes)

In [ ]:
print (df.shape)
df.head()

### 応答ベクターの作成

In [ ]:
y = df['churn'].values.astype(np.int)
y.size

###  余分な列の削除

In [ ]:
# df = df.drop(["Id","Churn"], axis = 1, inplace=True)
df.drop(["phone number","churn"], axis = 1, inplace=True)
df.head()

### feature matrixの作成

In [ ]:
X = df.values.astype(np.float)
print(X)
X.shape

### Feature Matrix 変数の正規化

In [ ]:
scaler = preprocessing.StandardScaler()
X = scaler.fit_transform(X)
X

## テストデータと学習データを分割する

モデルの作成フェーズでは、(層別交差検証を使用して) データセットをトレーニング・データとテスト・データに分割してから、個々の分類アルゴリズム (GradientBoostingClassifier、サポート・ベクトル・マシン、ランダム・フォレスト、k 近傍法など) を使用して複数のモデルをトレーニングしていきます。


In [ ]:
def stratified_cv(X, y, clf_class, shuffle=True, n_folds=10):
    stratified_k_fold = model_selection.StratifiedKFold(n_splits=n_folds, shuffle=shuffle)
    y_pred = y.copy()
    
    # ii -> train
    # jj -> test indices
    for ii, jj in stratified_k_fold.split(X, y): 
        X_train, X_test = X[ii], X[jj]
        y_train = y[ii]
        clf = clf_class
        clf.fit(X_train,y_train)
        y_pred[jj] = clf.predict(X_test)
    return y_pred

## モデルの作成

使用しているScikit-learnが搭載しているアルゴリズムを使用して、モデルを作成していきます。

In [ ]:
# create classifiers
from sklearn.ensemble import GradientBoostingClassifier
gradient_boost = GradientBoostingClassifier()

from sklearn.svm import SVC
svc_model = SVC(gamma='auto')

from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=10)

from sklearn.neighbors import KNeighborsClassifier
k_neighbors = KNeighborsClassifier()

from sklearn.linear_model import LogisticRegression
logistic_regression = LogisticRegression(solver='lbfgs')

In [ ]:
print('Gradient Boosting Classifier:  {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, gradient_boost))))
print('Support vector machine(SVM):   {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, svc_model))))
print('Random Forest Classifier:      {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, random_forest))))
print('K Nearest Neighbor Classifier: {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, k_neighbors))))
print('Logistic Regression:           {:.2f}'.format(metrics.accuracy_score(y, stratified_cv(X, y, logistic_regression))))

## モデルの評価

作成した種々のモデルの予測結果をマトリクスで描画し、それぞれの詳細を分析していきます。

### Gradient Boosting Classifier

In [ ]:
grad_ens_conf_matrix = metrics.confusion_matrix(y, stratified_cv(X, y, gradient_boost))
sns.heatmap(grad_ens_conf_matrix, annot=True,  fmt='');
title = 'Gradient Boosting'
plt.title(title);

### Support Vector Machines

In [ ]:
svm_svc_conf_matrix = metrics.confusion_matrix(y, stratified_cv(X, y, svc_model))
sns.heatmap(svm_svc_conf_matrix, annot=True,  fmt='');
title = 'SVM'
plt.title(title);

### Random Forest

In [ ]:
random_forest_conf_matrix = metrics.confusion_matrix(y, stratified_cv(X, y, random_forest))
sns.heatmap(random_forest_conf_matrix, annot=True,  fmt='');
title = 'Random Forest'
plt.title(title);

### Classification Report

In [ ]:
print('Gradient Boosting Classifier:\n {}\n'.format(metrics.classification_report(y, stratified_cv(X, y, gradient_boost))))
print('Support vector machine(SVM):\n {}\n'.format(metrics.classification_report(y, stratified_cv(X, y, svc_model))))
print('Random Forest Classifier:\n {}\n'.format(metrics.classification_report(y, stratified_cv(X, y, random_forest))))

## モデルの選択

今回の例では、Gradient Boostingアルゴリズムで作成したモデルを選択します。

In [ ]:
gbc = ensemble.GradientBoostingClassifier()
gbc.fit(X, y)

In [ ]:
# 本モデルにおいて、重要であると評価された属性を取得します。
feature_importance = gbc.feature_importances_
print (gbc.feature_importances_)
feat_importances = pd.Series(gbc.feature_importances_, index=df.columns)
feat_importances = feat_importances.nlargest(19)
feat_importances.plot(kind='barh' , figsize=(10,10)) 

## 【参考】Watson Machine Learning上にモデルを保存、デプロイする


AutoAIでモデルを作成したハンズオンでは、モデルの保存、WatsonMLへのデプロイはグラフィカルインターフェースで行いました。  
本Notebookのように、OSSを使用してモデルを作成した場合でも、もちろんWatsonMLへデプロイすることが可能です。  

保存先となるご自身のWatson MLサービスの資格情報を下記コードに記述すれば、上の手順で作成したモデルをデプロイすることが可能です。  
Watson MLとのやり取りはWatsonSDK（Watsonサービスとのやりとりを簡単に行えるライブラリ郡）を使用して行います。

In [ ]:
# create client to access our WML service
from watson_machine_learning_client import WatsonMachineLearningAPIClient

wml_credentials = {
  "apikey": "****",
  "iam_apikey_description": "Auto-generated for key ****",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/bc1bd51c396536dc7d5f81d5a4e****::serviceid:ServiceId-855c9709-e7f0-4fcc-8c48-93540cde****",
  "instance_id": "****",
  "password": "****",
  "url": "https://us-south.ml.cloud.ibm.com",
  "username": "****"
}

client = WatsonMachineLearningAPIClient(wml_credentials)
print(client.version)

In [ ]:
# Use this cell to do any cleanup of previously created models and deployments
client.repository.list_models()
client.deployments.list()

# client.repository.delete('GUID of stored model')
# client.deployments.delete('GUID of deployed model')

In [ ]:
# store the model in WML
meta_props={client.repository.ModelMetaNames.NAME: "Gradient Boosting model to predict customer churn"}
published_model = client.repository.store_model(model=gbc, meta_props={client.repository.ModelMetaNames.NAME: "Gradient Boosting model to predict customer churn"})


In [ ]:
# new list of models
client.repository.list_models()

# get UID of our just stored model
model_uid = client.repository.get_model_uid(published_model)
print("Model id: {}".format(model_uid))

In [ ]:
# create deployment
created_deployment = client.deployments.create(model_uid, name="customer_churn_model_gb")

# new list of deployments
client.deployments.list()

# get UID of our new deployment
deployment_uid = client.deployments.get_uid(created_deployment)
print("Deployment id: {}".format(deployment_uid))
print(created_deployment)

In [ ]:
# get scoring end point
scoring_endpoint = client.deployments.get_scoring_url(created_deployment)
print(scoring_endpoint)

In [ ]:
# use our WML client to score our model

# add some test data
scoring_payload = {'fields': ['state', 'account length', 'area code', 'international plan', 'voice mail plan',
        'number vmail messages', 'total day minutes', 'total day calls', 'total day charge', 'total eve minutes', 'total eve calls',
        'total eve charge', 'total night minutes', 'total night calls', 'total night charge', 'total intl minutes', 'total intl calls',
        'total intl charge', 'customer service calls'], 
                    'values': [[2,162,415,0,0,0,70.7,108,12.02,157.5,87,13.39,154.8,82,6.97,9.1,3,2.46,4]]}

In [ ]:
# score the model
predictions = client.deployments.score(scoring_endpoint, scoring_payload)
print('prediction',json.dumps(predictions, indent=2))

## 参考

本コードは以下を参考に作成されています。
https://www.ibm.com/developerworks/jp/library/watson-studio-using-jupyter-notebook/
（IBM Watson Studio無いでJupyterノートブックを使用してモデルを作成する）

https://www.kaggle.com/sandipdatta/customer-churn-analysis#

